# Digitalni Potpis sa RSA algoritmom

# Problem:
Kako možemo da osiguramo da poruka (ili fajl) dolazi zaista od određene osobe i da nije izmjenjena u međuvremenu? To radimo digitalnim potpisivanjem.

# Cilj:
- Generisati RSA ključeve (privatni i javni)
- Napraviti potpis teksta koristeći privatni ključ
- Verifikovati potpis koristeći javni ključ

# Potrebna biblioteka
%pip install pycryptodome


In [1]:
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256

In [2]:
# 1. Generisanje RSA ključeva

key = RSA.generate(2048)
private_key = key.export_key()
public_key = key.publickey().export_key()

# Snimamo ih u fajlove za kasniju upotrebu
with open("private.pem", "wb") as f:
    f.write(private_key)

with open("public.pem", "wb") as f:
    f.write(public_key)

print("Ključevi generisani i sačuvani kao 'private.pem' i 'public.pem'")

Ključevi generisani i sačuvani kao 'private.pem' i 'public.pem'


In [3]:
# 2. Pravimo test poruku koju ćemo potpisati

message = b"Ovo je veoma vazna poruka koju treba potpisati digitalno."

with open("message.txt", "wb") as f:
    f.write(message)

print("Poruka sačuvana u 'message.txt'")

Poruka sačuvana u 'message.txt'


In [7]:
# 3. Potpisivanje poruke privatnim ključem

# Učitavamo privatni ključ
with open("private.pem", "rb") as f:
    private_key = RSA.import_key(f.read())

# Učitavamo poruku
with open("message.txt", "rb") as f:
    message = f.read()

# Pravljenje hash-a poruke
hash_obj = SHA256.new(message)

# Potpisivanje
signature = pkcs1_15.new(private_key).sign(hash_obj)

# Čuvanje potpisa
with open("signature.sig", "wb") as f:
    f.write(signature)

print("Poruka potpisana i potpis sačuvan u 'signature.sig'")


Poruka potpisana i potpis sačuvan u 'signature.sig'


In [6]:
# 4. Verifikacija potpisa javnim ključem

# Učitavamo javni ključ
with open("public.pem", "rb") as f:
    public_key = RSA.import_key(f.read())

# Učitavamo potpis i poruku
with open("message.txt", "rb") as f:
    message = f.read()

with open("signature.sig", "rb") as f:
    signature = f.read()

# Pravimo hash poruke
hash_obj = SHA256.new(message)

# Verifikacija potpisa
try:
    pkcs1_15.new(public_key).verify(hash_obj, signature)
    print("Potpis je validan! Poruka nije izmjenjena.")
except (ValueError, TypeError):
    print("Nevalidan potpis! Poruka je izmjenjena ili nije od očekivanog pošiljaoca.")


Nevalidan potpis! Poruka je izmjenjena ili nije od očekivanog pošiljaoca.


# Objašnjenje:
Digitalni potpis radi tako što koristiš privatni ključ da potpišeš hash poruke.

Kasnije, bilo ko može da koristi javnim ključ da provjeri da li je potpis važeći i da poruka nije mijenjana.

Potpis se ne pravi nad čitavom porukom direktno, već nad njenim SHA-256 hash-om.

Ako se poruka promjeni, njen hash će se promjeniti, i verifikacija će pasti.

# Šta ovaj sistem obezbeđuje?
Autentičnost: potpis dokazuje da je pošiljalac imao privatni ključ.

Integritet: promjena poruke mijenja hash, pa verifikacija ne uspjeva.

Neporicanje (non-repudiation): pošiljalac ne može tvrditi da nije on poslao potpisanu poruku.

## Digitalni potpis
* Potpisuje dokument 
* Koristi privatni ključ 
* Dokazuje autentičnost i integritet podataka 
* Pravi ga vlasnik ključa 
* Može se verifikovati odmah 

## Digitalni certifikat
* Potvrđuje identitet
* Sadrži javni ključ
* Povezuje javni ključ sa identitetom
* Izdaje ga Certificate Authority 
* Moraš provjeriti da li je CA pouzdana